# Adding Limit Orders to the Trading Bot (Take Profit & Stop Loss)

## IBKR: Contrader Class without Take Profit & Stop Loss (Recap)

_Disclaimer: <br>
The following illustrative examples are for general information and educational purposes only. <br>
It is neither investment advice nor a recommendation to trade, invest or take whatsoever actions.<br>
The below code should only be used in combination with an IBKR Practice/Demo Account and NOT with a Live Trading Account._

In [ ]:
from ib_async import * 
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timezone # new
from IPython.display import display, clear_output
util.startLoop()

In [ ]:
def start_session():
    global last_update, session_start
    
    last_update = datetime.now(timezone.utc) # NEW 
    session_start = pd.to_datetime(last_update) # Updated (Python 3.12)
    
    initialize_stream()  
    stop_session()

def initialize_stream(): 
    global bars, last_bar
    
    bars = ib.reqHistoricalData(
            contract,
            endDateTime='',
            durationStr='1 D',
            barSizeSetting=freq,
            whatToShow='MIDPOINT',
            useRTH=True,
            formatDate=2,
            keepUpToDate=True)
    last_bar = bars[-1].date
    bars.updateEvent += onBarUpdate 
    
def onBarUpdate(bars, hasNewBar):  
    global df, last_bar, last_update
    
    last_update = datetime.now(timezone.utc) # NEW
    
    if bars[-1].date > last_bar: 
        last_bar = bars[-1].date
    
        # Data Processing
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1] 
        df.set_index("date", inplace = True)
        
        ####################### Trading Strategy ###########################
        df = df[["close"]].copy()
        df["returns"] = np.log(df["close"] / df["close"].shift())
        df["position"] = -np.sign(df.returns.rolling(window).mean())
        ####################################################################
        
        # Trading
        target = df["position"][-1] * units
        execute_trade(target = target)
        
        # Display
        clear_output(wait=True)
        display(df)
    else:
        try:
            trade_reporting()
        except:
            pass

def execute_trade(target):
    global current_pos
    
    # 1. get current Position
    try:
        current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
    except:
        current_pos = 0
         
    # 2. identify required trades
    trades = target - current_pos
        
    # 3. trade execution
    if trades > 0:
        side = "BUY"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)  
    elif trades < 0:
        side = "SELL"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    else:
        pass

def trade_reporting():
    global report
    
    fill_df = util.df([fs.execution for fs in ib.fills()])[["execId", "time", "side", "cumQty", "avgPrice"]].set_index("execId")
    profit_df = util.df([fs.commissionReport for fs in ib.fills()])[["execId", "realizedPNL"]].set_index("execId")
    report = pd.concat([fill_df, profit_df], axis = 1).set_index("time").loc[session_start:]
    report = report.groupby("time").agg({"side":"first", "cumQty":"max", "avgPrice":"mean", "realizedPNL":"sum"})
    report["cumPNL"] = report.realizedPNL.cumsum()
        
    clear_output(wait=True)
    display(df, report)

def stop_session():
    while True:
        ib.sleep(5) 
        if datetime.now(timezone.utc).time() >= end_time:
            execute_trade(target = 0) 
            ib.cancelHistoricalData(bars) 
            ib.sleep(10)
            try:
                trade_reporting() 
            except:
                pass
            print("Session Stopped (planned).")
            ib.disconnect()
            break
        elif datetime.now(timezone.utc) - last_update > dt.timedelta(seconds=120):
                # if there was no streaming update in the last 120 seconds
                ib.cancelHistoricalData(bars)
                ib.sleep(5)
                try: # try to reestablish stream
                    initialize_stream()
                except: # stop session
                    ib.sleep(5)
                    try:
                        execute_trade(target = 0) # close open position 
                    except:
                        pass
                    ib.sleep(10)
                    try:
                        trade_reporting() # final reporting
                    except:
                        pass
                    print("Session Stopped - No Connection.")
                    ib.disconnect()
                    break
        else:
            pass

In [ ]:
ib = IB()
ib.connect()

In [ ]:
# strategy parameters
freq = "1 min"
window = 1
units = 1000
end_time = (datetime.now(timezone.utc) + dt.timedelta(seconds = 330)).time() # stop condition (5.5 mins from now)
contract = Forex('EURUSD') 
ib.qualifyContracts(contract)
cfd = CFD("EUR", currency = "USD")
ib.qualifyContracts(cfd)
conID = cfd.conId

In [ ]:
start_session()

## IBKR: How to create Stop Loss and Take Profit Orders

In [ ]:
from ib_async import * 
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timezone # new
from IPython.display import display, clear_output
util.startLoop()

In [ ]:
ib = IB()
ib.connect()

In [ ]:
cfd = CFD("EUR", currency = "USD")
ib.qualifyContracts(cfd)
conID = cfd.conId

In [ ]:
def BracketOrder(parentOrderId, childOrderId1, childOrderId2,
                 action, quantity, stopLossPrice = None, takeProfitPrice = None):
    global stopLoss, takeProfit
    
    # Market Order (parent) - GO LONG or GO SHORT
    parent = Order()
    parent.orderId = parentOrderId
    parent.action = action
    parent.orderType = "MKT"
    parent.totalQuantity = quantity
    if not stopLossPrice and not takeProfitPrice: # if no sl/tp specified
        parent.transmit = True # transmit mkt order only
    else:
        parent.transmit = False
        
    bracketOrder = [parent]

    if stopLossPrice: # if sl specified
        # attached Stop Loss Order (child) 
        stopLoss = Order()
        stopLoss.orderId = childOrderId1
        stopLoss.action = "SELL" if action == "BUY" else "BUY"
        stopLoss.orderType = "STP"
        stopLoss.auxPrice = stopLossPrice
        stopLoss.totalQuantity = quantity
        stopLoss.parentId = parentOrderId
        if not takeProfitPrice: # if no tp specified
            stopLoss.transmit = True # transmit mkt order and sl order
        else:
            stopLoss.transmit = False
        bracketOrder.append(stopLoss)
    
    if takeProfitPrice:
        # attached Take Profit Order (child)
        takeProfit = Order()
        takeProfit.orderId = childOrderId2
        takeProfit.action = "SELL" if action == "BUY" else "BUY"
        takeProfit.orderType = "LMT"
        takeProfit.totalQuantity = quantity
        takeProfit.lmtPrice = takeProfitPrice
        takeProfit.parentId = parentOrderId
        takeProfit.transmit = True # transmit all three orders
        bracketOrder.append(takeProfit)
        
    return bracketOrder 

In [ ]:
# 1. BracketOrder with Mkt Order and SL (optional) and TP (optional)
bracket = BracketOrder(parentOrderId = ib.client.getReqId(), 
                       childOrderId1 = ib.client.getReqId(), 
                       childOrderId2 = ib.client.getReqId(),
                       action = "BUY",
                       quantity = 1000,
                       stopLossPrice = None, 
                       takeProfitPrice = None,
                      )
bracket

In [ ]:
# place all orders in BracketOrder
for o in bracket:
    ib.placeOrder(cfd, o)

In [ ]:
# close all positions/orders
try:
    ib.cancelOrder(stopLoss)
except:
    pass
try:
    ib.cancelOrder(takeProfit)
except:
    pass
order = MarketOrder("Sell", 1000)
ib.placeOrder(cfd, order)

## IBKR: Stop Loss and Take Profit - Pitfalls and other Considerations

__Be careful__: 
- SL Price must be below (above) Market Entry Price in Long (Short) position
- TP Price must be above (below) Market Entry Price in Long (Short) position
- Stop Loss and Take Profit Prices must be limited to [5] decimals [EUR/USD]

__Setting Stop Loss Price and Take Profit Price__

Typically: Define Maximum Loss & Take Profit in % (e.g. 1% loss -> 10% if leverage == 10)

In [ ]:
sl_perc = 0.01 # SL Event if Price moves into the wrong direction by 1%
tp_perc = 0.01 # TP Event if Price moves into the right direction by 1%

In [ ]:
current_price = 1.10

In [ ]:
target = -1 # (Target Position: -1/0/1)

In [ ]:
if sl_perc:
        if target > 0: # LONG
            sl_price = round(current_price * (1 - sl_perc), 4) 
        elif target < 0: # SHORT
            sl_price = round(current_price * (1 + sl_perc), 4)      
        else: 
            sl_price = None

if tp_perc:
        if target > 0: # LONG
            tp_price = round(current_price * (1 + tp_perc), 4) 
        elif target < 0: # SHORT
            tp_price = round(current_price * (1 - tp_perc), 4)      
        else: 
            tp_price = None

In [ ]:
sl_price

In [ ]:
tp_price

__Identification of SL/TP Events__

Problem in Trading Bot: it doesn´t record TP/SL trades

Solution: Check if expected trading position == actual trading position (if not: very likely SL/TP Event)

## IBKR: Adding Stop Loss and Take Profit to the Class

In [ ]:
from ib_async import * 
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timezone # new
from IPython.display import display, clear_output
util.startLoop()

In [ ]:
def start_session():
    global last_update, session_start, exp_pos, current_pos
    
    exp_pos = 0 # Added
    current_pos = 0 # Added
    last_update = datetime.now(timezone.utc) # NEW 
    session_start = pd.to_datetime(last_update) # Updated (Python 3.12)
    
    initialize_stream()  
    stop_session()

def initialize_stream(): 
    global bars, last_bar
    
    bars = ib.reqHistoricalData(
            contract,
            endDateTime='',
            durationStr='1 D',
            barSizeSetting=freq,
            whatToShow='MIDPOINT',
            useRTH=True,
            formatDate=2,
            keepUpToDate=True)
    last_bar = bars[-1].date
    bars.updateEvent += onBarUpdate 
    
def onBarUpdate(bars, hasNewBar):  
    global df, last_bar, last_update
    
    last_update = datetime.now(timezone.utc) # NEW
    
    if bars[-1].date > last_bar: 
        last_bar = bars[-1].date
    
        # Data Processing
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1] 
        df.set_index("date", inplace = True)
        
        ####################### Trading Strategy ###########################
        df = df[["close"]].copy()
        df["returns"] = np.log(df["close"] / df["close"].shift())
        df["position"] = -np.sign(df.returns.rolling(window).mean())
        ####################################################################
        
        # Trading
        target = df["position"][-1] * units
        execute_trade(target = target)
        
        # Display
        clear_output(wait=True)
        display(df)
    else:
        try:
            trade_reporting()
        except:
            pass

def execute_trade(target): # Modified!!!
    global exp_pos
         
    # 1. identify required trades
    trades = target - exp_pos
    
    # 2. determine SL Price and TP Price
    current_price = df.close.iloc[-1]

    if sl_perc:
        if target > 0: # LONG
            sl_price = round(current_price * (1 - sl_perc), 4) 
        elif target < 0: # SHORT
            sl_price = round(current_price * (1 + sl_perc), 4)      
    else: 
        sl_price = None

    if tp_perc:
        if target > 0: # LONG
            tp_price = round(current_price * (1 + tp_perc), 4) 
        elif target < 0: # SHORT
            tp_price = round(current_price * (1 - tp_perc), 4)      
    else: 
        tp_price = None
        
    # 3. trade execution
    if target > 0: # GOING LONG
        if current_pos == 0: # from NEUTRAL
            go_long_short(side = "BUY", target = target, sl_price = sl_price, tp_price = tp_price) 
        elif current_pos < 0: # from SHORT:
            cancel_orders() # cancel sl/tp orders
            go_neutral(side = "BUY", trades = current_pos)
            go_long_short(side = "BUY", target = target, sl_price = sl_price, tp_price = tp_price)
    elif target < 0: # GOING SHORT
        if current_pos == 0: # from NEUTRAL  
            go_long_short(side = "SELL", target = abs(target), sl_price = sl_price, tp_price = tp_price) 
        elif current_pos > 0: # from LONG
            cancel_orders() # cancel sl/tp orders
            go_neutral(side = "SELL", trades = current_pos)
            go_long_short(side = "SELL", target = abs(target), sl_price = sl_price, tp_price = tp_price)
    else: # GOING NEUTRAL
        if current_pos < 0: # from SHORT
            cancel_orders() # cancel sl/tp orders
            go_neutral(side = "BUY", trades = current_pos)
        elif current_pos > 0: # from LONG:
            cancel_orders() # cancel sl/tp orders
            go_neutral(side = "SELL", trades = current_pos)
    exp_pos = target

def go_long_short(side, target, sl_price, tp_price): # NEW Go Long/Short starting from Neutral posistion
    bracket = BracketOrder(parentOrderId = ib.client.getReqId(), 
                           childOrderId1 = ib.client.getReqId(), 
                            childOrderId2 = ib.client.getReqId(),
                            action = side,
                            quantity = target,
                            stopLossPrice = sl_price, 
                            takeProfitPrice = tp_price,
                          )
    for o in bracket:
        order = ib.placeOrder(cfd, o)
    
def go_neutral(side, trades): # Close Long/Short position
    order = MarketOrder(side, abs(trades))
    trade = ib.placeOrder(cfd, order)    
    
def cancel_orders(): # cancel SL/TP orders
    try:
        sl_cancel = ib.cancelOrder(stopLoss)
    except:
        pass
    try:
        tp_cancel = ib.cancelOrder(takeProfit)
    except:
        pass 

def BracketOrder(parentOrderId, childOrderId1, childOrderId2,
                 action, quantity, stopLossPrice, takeProfitPrice): # NEW
    global stopLoss, takeProfit
    
    # Market Order (parent) - GO LONG or GO SHORT
    parent = Order()
    parent.orderId = parentOrderId
    parent.action = action
    parent.orderType = "MKT"
    parent.totalQuantity = quantity
    if not stopLossPrice and not takeProfitPrice: 
        parent.transmit = True
    else:
        parent.transmit = False
        
    bracketOrder = [parent]

    if stopLossPrice:
        # attached Stop Loss Order (child) 
        stopLoss = Order()
        stopLoss.orderId = childOrderId1
        stopLoss.action = "SELL" if action == "BUY" else "BUY"
        stopLoss.orderType = "STP"
        stopLoss.auxPrice = stopLossPrice
        stopLoss.totalQuantity = quantity
        stopLoss.parentId = parentOrderId
        if not takeProfitPrice: 
            stopLoss.transmit = True
        else:
            stopLoss.transmit = False
        bracketOrder.append(stopLoss)
    
    if takeProfitPrice:
        # attached Take Profit Order (child)
        takeProfit = Order()
        takeProfit.orderId = childOrderId2
        takeProfit.action = "SELL" if action == "BUY" else "BUY"
        takeProfit.orderType = "LMT"
        takeProfit.totalQuantity = quantity
        takeProfit.lmtPrice = takeProfitPrice
        takeProfit.parentId = parentOrderId
        takeProfit.transmit = True
        bracketOrder.append(takeProfit)
        
    return bracketOrder 
    
def trade_reporting():
    global report
    
    fill_df = util.df([fs.execution for fs in ib.fills()])[["execId", "time", "side", "shares", "avgPrice"]].set_index("execId")
    profit_df = util.df([fs.commissionReport for fs in ib.fills()])[["execId", "realizedPNL"]].set_index("execId")
    report = pd.concat([fill_df, profit_df], axis = 1).set_index("time").loc[session_start:]
    report = report.groupby(["time", "side"]).agg({"shares":"sum", "avgPrice":"mean", "realizedPNL":"sum"}).reset_index().set_index("time")
    report["cumPNL"] = report.realizedPNL.cumsum()
        
    clear_output(wait=True)
    display(df, report)

def stop_session(): # Third case added
    global current_pos
    
    while True:
        ib.sleep(5) 
        try:
            current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
        except:
            current_pos = 0
        if datetime.now(timezone.utc).time() >= end_time:
            execute_trade(target = 0) 
            ib.cancelHistoricalData(bars) 
            ib.sleep(10)
            try:
                trade_reporting() 
            except:
                pass
            print("Session Stopped (planned).")
            ib.disconnect()
            break
        elif exp_pos != current_pos: # if SL/TP Event
            ib.sleep(5)
            try:
                current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
            except:
                current_pos = 0
            if exp_pos != current_pos:
                execute_trade(target = 0) 
                ib.cancelHistoricalData(bars) 
                ib.sleep(10)
                try:
                    trade_reporting() 
                except:
                    pass
                print("Session Stopped (SL/TP Event).")
                ib.disconnect()
                break
            else:
                pass
        elif datetime.now(timezone.utc) - last_update > dt.timedelta(seconds=120):
                # if there was no streaming update in the last 120 seconds
                ib.cancelHistoricalData(bars)
                ib.sleep(5)
                try: # try to reestablish stream
                    initialize_stream()
                except: # stop session
                    ib.sleep(5)
                    try:
                        execute_trade(target = 0) # close open position 
                    except:
                        pass
                    ib.sleep(10)
                    try:
                        trade_reporting() # final reporting
                    except:
                        pass
                    print("Session Stopped - No Connection.")
                    ib.disconnect()
                    break
        else:
            pass

In [ ]:
ib = IB()
ib.connect()

In [ ]:
# strategy parameters
freq = "1 min"
window = 1
units = 1000
end_time = (datetime.now(timezone.utc) + dt.timedelta(seconds = 330)).time() # stop condition (5.5 mins from now)
sl_perc = 0.1
tp_perc = 0.1
contract = Forex('EURUSD') 
ib.qualifyContracts(contract)
cfd = CFD("EUR", currency = "USD")
ib.qualifyContracts(cfd)
conID = cfd.conId

In [ ]:
start_session()